## Desenvolvimento do Modelo de Machine Learning (Regressão)

### Carregamento dos dados

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

O csv carregado abaixo é o resultado do pré-processamento e AED feita no notebook 2_EDA_e_preprocessamento.ipynb

In [100]:
imoveis = pd.read_csv('casas_zapimoveis_clean.csv')
imoveis.head()

,endereco,preco,area,quartos,banheiros,garagens,latitude,longitude
0,"Plano Diretor Norte, Palmas",1000000,112,3,2,4,-10.170166,-48.333980
1,"ARSO 43 Alameda 3, Plano Diretor Sul",625000,150,3,2,2,-10.215773,-48.334077
2,"Quadra ARNE 64 Alameda 2, Plano Diretor Norte",315000,115,3,1,3,-10.156889,-48.313459
3,"Plano Diretor Sul, Palmas",220000,86,2,2,2,-10.219647,-48.338964
4,"Quadra ARSE 92 Alameda 4, Plano Diretor Sul",550000,190,4,2,2,-10.215587,-48.346005


### Definição das entradas e saídas do modelo

In [101]:
X = imoveis.drop(['preco', 'endereco'], axis=1)
y = imoveis['preco']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Padronização (scaling) das entradas

In [102]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Regressão Linear

O primeiro modelo que vamos criar é de Regressão Linear simples:

In [103]:
lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)
lin_y_pred = lin_reg.predict(X_test)

# Guardamos as métricas para depois avaliar qual é o melhor modelo
lin_rmse = mean_squared_error(y_test, lin_y_pred, squared=False)
lin_score_treino = lin_reg.score(X_train, y_train)
lin_score_teste = lin_reg.score(X_test, y_test)

### Regressão Ridge

O segundo modelo que criamos é baseado na Regressão Rigde, que previne overfitting. Utilizamos Cross Validation para determinar o melhor parâmetro alpha do modelo.

In [151]:
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

ridge = RidgeCV(alphas=np.arange(0.1, 20, 0.1), cv=cv, scoring='neg_mean_absolute_error')
ridge.fit(X_train, y_train)
ridge_y_pred = ridge.predict(X_test)

# Guardamos as métricas para depois avaliar qual é o melhor modelo
ridge_rmse = mean_squared_error(y_test, ridge_y_pred, squared=False)
ridge_score_treino = ridge.score(X_train, y_train)
ridge_score_teste = ridge.score(X_test, y_test)

Obs.: Após testes verifiquei que aumentar o limite máximo do alpha nesse CV leva a uma melhora insignificante nas métricas, por isso mantive o valor máximo em 20.

### Regressão Lasso

O terceiro modelo que criamos é baseado na Regressão Lasso, que também previne overfitting. Utilizamos Cross Validation para determinar o melhor parâmetro alpha do modelo.

In [129]:
lasso = LassoCV(alphas=np.arange(0.1, 20, 0.1), cv=cv)
lasso.fit(X_train, y_train)
lasso_y_pred = lasso.predict(X_test)

lasso_rmse = mean_squared_error(y_test, lasso_y_pred, squared=False)
lasso_score_treino = lasso.score(X_train, y_train)
lasso_score_teste = lasso.score(X_test, y_test)

Obs.: Após testes verifiquei que aumentar o limite máximo do alpha nesse CV leva a uma melhora insignificante nas métricas, por isso mantive o valor máximo em 20.

### Comparando os resultados

In [130]:
results = pd.DataFrame({
    'Score Treino': [lin_score_treino, ridge_score_treino, lasso_score_treino],
    'Score Teste': [lin_score_teste, ridge_score_teste, lasso_score_teste],
    'RMSE': [lin_rmse, ridge_rmse, lasso_rmse]
}, index=['Linear', 'Ridge', 'Lasso'])
results

,Score Treino,Score Teste,RMSE
Linear,0.540432,0.482927,366122.260083
Ridge,0.540331,0.483416,365949.054709
Lasso,0.540432,0.482924,366123.180761


### Escolhendo e Salvando o Melhor Modelo

Como vimos acima, o modelo que desempenhou melhor foi a Regressão Ridge, tanto no score de teste como no RMSE. Portanto é o modelo Ridge que vamos salvar para utilizar na predição de novos imóveis (arquivo 4_preditor_de_precos.ipynb)

In [131]:
print("O modelo escolhido (Ridge) tem estes parâmetros:")
print("Alpha: ", ridge.alpha_)
print("Coeficientes: ", ridge.coef_)
print("Interseção: ", ridge.intercept_)

O modelo escolhido (Ridge) tem estes parâmetros:
Alpha:  19.900000000000002
Coeficientes:  [183106.1491094   55557.46152389 174866.14434879   5257.61589643
  13773.25544349 -46459.01225472]
Interseção:  722998.5971564258


Salvando o modelo Ridge em um arquivo joblib, assim como o scaler:

In [135]:
_ = joblib.dump(ridge, 'ridge_final.model')

# Para utilizar o modelo Ridge precisamos padronizar a entrada, por isso precisamos gravar também o scaler treinado
_ = joblib.dump(scaler, 'padronizador_entrada.scaler')

['padronizador_entrada.scaler']